In [6]:
import pandas as pd
import glob
import xarray as xr
import numpy as np

In [2]:
ordered_list = ['frontalpole-rh', 'frontalpole-lh', 'rostralmiddlefrontal-rh', 'rostralmiddlefrontal-lh', 'parsorbitalis-rh',
                'parsorbitalis-lh', 'medialorbitofrontal-rh', 'medialorbitofrontal-lh', 'rostralanteriorcingulate-rh', 
                'rostralanteriorcingulate-lh', 'parstriangularis-rh', 'parstriangularis-lh', 'lateralorbitofrontal-rh', 
                'lateralorbitofrontal-lh', 'superiorfrontal-rh', 'superiorfrontal-lh', 'caudalanteriorcingulate-rh',
                'caudalanteriorcingulate-lh', 'parsopercularis-rh', 'parsopercularis-lh', 'caudalmiddlefrontal-rh', 
                'caudalmiddlefrontal-lh', 'temporalpole-rh', 'temporalpole-lh', 'insula-rh', 'insula-lh', 'entorhinal-rh', 
                'entorhinal-lh', 'precentral-rh', 'precentral-lh', 'posteriorcingulate-rh', 'posteriorcingulate-lh', 
                'superiortemporal-rh', 'superiortemporal-lh', 'postcentral-rh', 'postcentral-lh', 'transversetemporal-rh', 
                'transversetemporal-lh', 'paracentral-rh', 'paracentral-lh', 'middletemporal-rh', 'middletemporal-lh', 
                'parahippocampal-rh', 'parahippocampal-lh', 'inferiortemporal-rh', 'inferiortemporal-lh', 'supramarginal-rh', 
                'supramarginal-lh', 'fusiform-rh', 'fusiform-lh', 'bankssts-rh', 'bankssts-lh', 'isthmuscingulate-rh', 
                'isthmuscingulate-lh', 'precuneus-rh', 'precuneus-lh', 'superiorparietal-rh', 'superiorparietal-lh', 
                'inferiorparietal-rh', 'inferiorparietal-lh', 'lingual-rh', 'lingual-lh', 'cuneus-rh', 'cuneus-lh', 
                'pericalcarine-rh', 'pericalcarine-lh', 'lateraloccipital-rh', 'lateraloccipital-lh']

In [3]:
brainder_list = []
for i in range(len(ordered_list)):
    name = ordered_list[i].split('-')[1] + '.pial.DK' + '.' + ordered_list[i].split('-')[0]
    brainder_list.append(name)

In [8]:
methods = ['dynsys']
conditions = ['EC','EO']

In [ ]:
#mean and std matrix generation

In [18]:
for method in methods:
    for condition in conditions:
        xarrays = glob.glob(f'/scratch/{method}/*{method}_{condition}_alpha.nc')
        
        xarrays_combined = []
        subjects = []
        for array in xarrays:
            subject = array.split('/')[3].split('_')[0]
            subjects.append(subject)
            xarrays_combined.append(xr.open_dataarray(array))
        
        xarrays_combined = xr.concat(xarrays_combined, pd.Index(subjects, name="subjects"))
        
        mean_df = xarrays_combined.mean(dim=["bootstrap_samples","subjects"]).values
        mean_df = pd.DataFrame(mean_df)
        mean_df.columns = xarrays_combined.region1.values
        mean_df.index = xarrays_combined.region1.values

        ordered_mean_df = []
        for region in ordered_list:
            ordered_mean_df.append(mean_df[region].values)
    
        ordered_mean_df = pd.DataFrame(np.array(ordered_mean_df))
    
        ordered_mean_df.columns = ordered_list
        ordered_mean_df.index = xarrays_combined.region1.values

        ordered_mean_df_2 = []
        for region in ordered_list:
            ordered_mean_df_2.append(ordered_mean_df[region].values)
    
        ordered_mean_df_2 = pd.DataFrame(ordered_mean_df_2)
        ordered_mean_df_2.index = brainder_list
        ordered_mean_df_2.columns = brainder_list
        
        ordered_mean_df_2.to_csv(f'/scratch/{method}_{condition}_mean_df.csv',header=True,index=False)
        
        std_df = xarrays_combined.std(dim=["bootstrap_samples","subjects"]).values 
        std_df = pd.DataFrame(std_df)
        std_df.columns = xarrays_combined.region1.values
        std_df.index = xarrays_combined.region1.values
        
        ordered_std_df = []
        for region in ordered_list:
            ordered_std_df.append(std_df[region].values)
    
        ordered_std_df = pd.DataFrame(np.array(ordered_std_df))
    
        ordered_std_df.columns = ordered_list
        ordered_std_df.index = xarrays_combined.region1.values

        ordered_std_df_2 = []
        for region in ordered_list:
            ordered_std_df_2.append(ordered_std_df[region].values)
    
        ordered_std_df_2 = pd.DataFrame(np.array(ordered_std_df_2))
    
        ordered_std_df_2.columns = brainder_list
        ordered_std_df_2.index = brainder_list
        
        ordered_std_df_2.to_csv(f'/scratch/{method}_{condition}_std_df.csv',header=True,index=False)

In [ ]:
#cohens_d calculation

In [46]:
for method in methods:
    mean_ec = pd.read_csv(f'/scratch/{method}_EC_mean_df.csv')
    mean_eo = pd.read_csv(f'/scratch/{method}_EO_mean_df.csv')
    std_ec = pd.read_csv(f'/scratch/{method}_EC_std_df.csv')
    std_eo = pd.read_csv(f'/scratch/{method}_EO_std_df.csv')
    
    len_ec = 67200
    len_eo = 67200
    
    pooled_sd = np.sqrt(((len_ec-1)*(std_ec**2) + (len_eo-1)*(std_eo**2))/(len_ec + len_eo - 2))
    
    cohens_d = (mean_ec - mean_eo)/pooled_sd
    cohens_d.to_csv(f'/scratch/{method}_cohens_d.csv',header=True,index=False)

In [ ]:
#visual

In [ ]:
ordered_list_visual =['fusiform-rh', 'fusiform-lh', 'lingual-rh', 'lingual-lh', 'cuneus-rh', 'cuneus-lh',
                      'lateraloccipital-rh', 'lateraloccipital-lh']

In [ ]:
for method in methods:
    for condition in conditions:
        xarrays = glob.glob(f'/scratch/{method}/*{method}_{condition}_alpha.nc')
        
        xarrays_combined = []
        subjects = []
        for array in xarrays:
            subject = array.split('/')[3].split('_')[0]
            subjects.append(subject)
            xarrays_combined.append(xr.open_dataarray(array))
        
        xarrays_combined = xr.concat(xarrays_combined, pd.Index(subjects, name="subjects"))
        
        xarrays_combined = xarrays_combined.sel(region1=ordered_list_visual,region2=ordered_list_visual)
        mean_df = xarrays_combined.mean(dim=["bootstrap_samples","subjects"]).values
        mean_df = pd.DataFrame(mean_df)
        mean_df.columns = xarrays_combined.region1.values
        mean_df.index = xarrays_combined.region1.values

        mean_df.to_csv(f'/scratch/{method}_{condition}_mean_df_visual.csv',header=True,index=False)

        std_df = xarrays_combined.std(dim=["bootstrap_samples","subjects"]).values
        std_df = pd.DataFrame(std_df)
        std_df.columns = xarrays_combined.region1.values
        std_df.index = xarrays_combined.region1.values
        
        std_df.to_csv(f'/scratch/{method}_{condition}_std_df_visual.csv',header=True,index=False)

In [ ]:
for method in methods:
    mean_ec = pd.read_csv(f'/scratch/{method}_EC_mean_df_visual.csv')
    mean_eo = pd.read_csv(f'/scratch/{method}_EO_mean_df_visual.csv')
    std_ec = pd.read_csv(f'/scratch/{method}_EC_std_df_visual.csv')
    std_eo = pd.read_csv(f'/scratch/{method}_EO_std_df_visual.csv')
    
    len_ec = 67200
    len_eo = 67200
    
    pooled_sd = np.sqrt(((len_ec-1)*(std_ec**2) + (len_eo-1)*(std_eo**2))/(len_ec + len_eo - 2))
    
    cohens_d = (mean_ec - mean_eo)/pooled_sd
    cohens_d.to_csv(f'/scratch/{method}_cohens_d_visual.csv',header=True,index=False)

In [ ]:
#dmn

In [ ]:
ordered_list_dmn = ['medialorbitofrontal-rh','medialorbitofrontal-lh','rostralanteriorcingulate-rh','rostralanteriorcingulate-lh',
                    'lateralorbitofrontal-rh','lateralorbitofrontal-lh','parahippocampal-rh','parahippocampal-lh','posteriorcingulate-rh',
                    'posteriorcingulate-lh','isthmuscingulate-rh','isthmuscingulate-lh','precuneus-rh','precuneus-lh']

In [ ]:
for method in methods:
    for condition in conditions:
        xarrays = glob.glob(f'/scratch/{method}/*{method}_{condition}_alpha.nc')
        
        xarrays_combined = []
        subjects = []
        for array in xarrays:
            subject = array.split('/')[3].split('_')[0]
            subjects.append(subject)
            xarrays_combined.append(xr.open_dataarray(array))
        
        xarrays_combined = xr.concat(xarrays_combined, pd.Index(subjects, name="subjects"))
        
        xarrays_combined = xarrays_combined.sel(region1=ordered_list_dmn,region2=ordered_list_dmn)
        mean_df = xarrays_combined.mean(dim=["bootstrap_samples","subjects"]).values
        mean_df = pd.DataFrame(mean_df)
        mean_df.columns = xarrays_combined.region1.values
        mean_df.index = xarrays_combined.region1.values

        mean_df.to_csv(f'/scratch/{method}_{condition}_mean_df_dmn.csv',header=True,index=False)

        std_df = xarrays_combined.std(dim=["bootstrap_samples","subjects"]).values
        std_df = pd.DataFrame(std_df)
        std_df.columns = xarrays_combined.region1.values
        std_df.index = xarrays_combined.region1.values
        
        std_df.to_csv(f'/scratch/{method}_{condition}_std_df_dmn.csv',header=True,index=False)

In [10]:
for method in methods:
    for condition in conditions:
        mean_ec = pd.read_csv(f'/scratch/{method}_EC_mean_df_dmn.csv')
        mean_eo = pd.read_csv(f'/scratch/{method}_EO_mean_df_dmn.csv')
        std_ec = pd.read_csv(f'/scratch/{method}_EC_std_df_dmn.csv')
        std_eo = pd.read_csv(f'/scratch/{method}_EO_std_df_dmn.csv')
        
        len_ec = 67200
        len_eo = 67200
        
        pooled_sd = np.sqrt(((len_ec-1)*(std_ec**2) + (len_eo-1)*(std_eo**2))/(len_ec + len_eo - 2))
        
        cohens_d = (mean_ec - mean_eo)/pooled_sd
        cohens_d.to_csv(f'/scratch/{method}_cohens_d_dmn.csv',header=True,index=False)

FileNotFoundError: [Errno 2] No such file or directory: '/scratch/dynsys_EC_mean_df_dmn.csv'